#Структурные шаблоны

Декоратор (Decorator) — это шаблон, который позволяет добавлять объектам новую функциональность, как бы оборачивая их.

К счастью разработчиков на Python, данный шаблон встроен по умолчанию. 

Более подробно о работе с декораторами можно посмотреть в модуле «Функции». Тут же еще раз приведем пример работы с декораторами в Python.

In [4]:
def func_time_decorator(func):
  import time
  def wrapper(*args, **kwargs):
    start = time.time()
    return_value = func(*args, **kwargs)
    end = time.time()
    print('[*] Время выполнения: {} секунд.'.format(end-start))
    return return_value
  return wrapper

__Фасад (Facade) — это шаблон, который предоставляет интерфейс к сложной системе классов, сторонних библиотек и фреймворков.__

Пример: подключение нового функционала библиотеки Prometeus (для мониторинга), Jaeger (для трассировки) удобно делать с помощью классов, которые изолируют вызов и работу с этой библиотекой, отделяя ее от классов разрабатываемой системы.

In [9]:
class FileAdapterFacade:
  """
  Фасад для доступа к файлам, который скрывает
  конкретную систему, где лежат файлы
  """
  def __init__(self, subsystem) -> None:
    self._file_system = subsystem
  def get_file(self, file_name):
    return self._file_system.get(file_name)

class LocalStorager:
  """
  Доступ к файлам на диске
  """
  def get_file(self, file_name):
    pass

class CephStorager:
  """
  Доступ к файлам, которые лежат удаленно в ceph bucket
  Данный класс вдобавок может реализовывать
  работы с библиотекой, подключения к ceph
  """ 
  def get_file(self, file_name):
    connector = self.connect()
    connector.set_s3_mode()
    connector.turn_off_ssl()
    connector.estabilish()
    file = connector.get_file(file_name)
    return file

__Адаптер (Adapter) — это шаблон, который позволяет объектам с разными интерфейсами работать вместе.__

Пример: часто при разработке нескольких систем приходится разрабатывать систему-адаптер, которая через нее связывает несколько систем в единую. Внутри одного проекта это может быть класс, который готовит данные, а отдельные классы адаптируют их для отправки по rest или по kafka.

In [13]:
# coding=utf-8
import requests

CORRELATION_ID = "correlationId"

class HttpRequest:
  def __init__(self, url):
    self.url = url

  def request(self, obj):
    try:
      if CORRELATION_ID not in obj.headers.keys():
        raise AttributeError
      else:
        response = requests.get(
            self.url,
            params=obj.params,
            headers=obj.headers,
        )
        print(response)
    except AttributeError:
      print("not correlationId in headers")

class OldClass:
  def __init__(self, headers, params):
    self.headers = headers
    self.params = params

class OldClassHttpRequestAdapter:
  def __init__(self, obj):
    self.obj = obj
  @property
  def headers(self):
    if CORRELATION_ID not in self.obj.headers.keys():
      self.obj.headers[CORRELATION_ID] = "1232-1112-3333"
    return self.obj.headers
  @property
  def params(self):
    return self.obj.params

http_request = HttpRequest("https://github.com")
old_class_obj = OldClass({"test":"111"}, {"params1":"value1"})
adapter_obj = OldClassHttpRequestAdapter(old_class_obj)
http_request.request(old_class_obj)
http_request.request(adapter_obj)

not correlationId in headers
<Response [200]>


__Компоновщик (Composite) — это шаблон, который позволяет сгруппировать множество объектов в дерево, а затем работать с ней, как с единым объектом.__

Пример: при разработке программы под Windows / Mac с GUI, графический интерфейс можно представить в виде дерева, который с помощью компоновщика можно собрать в единое целое.

In [17]:
from abc import ABC, abstractmethod

class GUIAbstractComponent(ABC):
  """
  Базовый класс абстрактный компонент GUI интерфейса 
  """
  @property
  def parent(self):
    return self._parent
  @parent.setter
  def parent(self, parent):
    self._parent = parent
  def add(self, component):
    pass
  def remove(self, component):
    pass
  @abstractmethod
  def action(self):
    pass

class Button(GUIAbstractComponent):
  """
  Не имеет вложенных компонентов, выполняет действия.
  """
  def action(self):
    return "Button"

class EditBox(GUIAbstractComponent):
  def action(self):
    return "Leaf"

class Window(GUIAbstractComponent):
  """
  Контейнер, который содержить в себе более простые объекты
  """
  def __init__(self):
    self._children = []
  def add(self, component):
    self._children.append(component)
    component.parent = self
  def remove(self, component):
    self._cheldren.remove(component)
    component.parent = None
  def is_composite(self):
    return True
  def action(self):
    """
    Контейнер проходится по своим детям и вызывает у них
    действия, суммирует и отдает результат.
    Таким образом, можно организовать перерисовку
    всех компонентов в GUI.
    """
    results = []
    for child in self._children:
      results.append(child.action())
    return f"Branch({'+'.join(results)})"

if __name__ == "__main__":
  window = Window()
  window.add(Button())
  branch1.add(EditBox())

__Заместитель (Proxy) — это шаблон, который позволяет вместо реальных объектов передать «заместителей», которые позволяют перехватить обращение и выполнить действие до и после вызова.__

Пример: вы используете библиотеку, которая реализовала доступ какому-нибудь ресурсу вашей сети, но в ней нет проверки на доступность этого ресурса. Вместо переписывания своего кода можно заменить объект класса из библиотеки собственным классом-заместителем, который реализует тот же самый метод, но перед этим собственными силами проверяет доступность ресурса в сети.

In [18]:
class LocalWeb():
  def request(self):
    print("request to locacweb")
  
class Proxy(LocalWeb):
  """
  Наследник от класса из библиотеки
  """
  def __init__(self, local_web):
    self._local_web = local_web
  def request(self):
    if self.check.access():
      self._local_web.request()
  def check_access(self) -> bool:
    return True